In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
df=pd.read_csv("./data/selected_data_ISONE.csv")
df

,date,year,month,day,weekday,hour,demand,temperature
0,2003/3/1,2003,3,1,7,1,12863.0,23
1,2003/3/1,2003,3,1,7,2,12389.0,22
2,2003/3/1,2003,3,1,7,3,12155.0,21
3,2003/3/1,2003,3,1,7,4,12072.0,21
4,2003/3/1,2003,3,1,7,5,12160.0,22
5,2003/3/1,2003,3,1,7,6,12568.0,21
6,2003/3/1,2003,3,1,7,7,13236.0,22
7,2003/3/1,2003,3,1,7,8,14190.0,22
8,2003/3/1,2003,3,1,7,9,15213.0,24
9,2003/3/1,2003,3,1,7,10,15647.0,27


In [3]:
len(df)

103776

In [4]:
def preprocess(df):
    df.dropna(axis=1,inplace=True)
    y=list()
    for i in range(len(df)):
        y.append(df.loc[i,'demand'])
    
    y=np.array(y)
    
    x=list()
    for i in range(len(df)):
        x.append(df.loc[i,'day'])
        x.append(df.loc[i,'month'])
        x.append(df.loc[i,'year'])
        x.append(df.loc[i,'weekday'])
        x.append(df.loc[i,'hour'])
        x.append(df.loc[i,'temperature'])
    
    x=np.array(x)
    x=x.reshape(-1,6)
    return x,y

In [5]:
x,y=preprocess(df)
print(x.shape,y.shape)

(103776, 6) (103776,)


In [6]:
import pickle
def save_data(x,y,s1,s2):
    pic_out=open("./data/"+s1+".pickle","wb")
    pickle.dump(x,pic_out)
    pic_out.close()

    pic_out=open("./data/"+s2+".pickle","wb")
    pickle.dump(y,pic_out)
    pic_out.close()

In [7]:
save_data(x,y,'x','y')

In [8]:
def train_test_split(x,y,time_step=1):
    x=np.hstack((x[1:],y[:-1].reshape(-1,1))) #adding previous_hour_load as a new feature
    
    #scaling between 0 to 1
    x=MinMaxScaler(feature_range=(0,1)).fit_transform(x)
    
    #adding time step
    from collections import deque
    prev=deque(maxlen=time_step)
    for i in range(time_step-1):
        prev.append(list(x[i]))

    xx=list()
    for i in range(time_step-1,len(x)):
        prev.append(list(x[i]))
        xx.append(list(prev))
        
    #corresponding output data
    y=y[time_step:]
    
    xx=np.array(xx)
    #scaling the output (demand of current hour)
    scale=MinMaxScaler(feature_range=(0,1))
    y=scale.fit_transform(y.reshape(-1,1))
    #minmaxscaler formula=(x-xmin)/(xmax-xmin)
    
    #input shape (sample,time step, feature)
    #splitting in train and test set.93% as train and 7% as test data
    train_test_split=int(len(xx)*0.93)
    x_test=xx[train_test_split:]
    x_train=xx[:train_test_split]
    y_test=y[train_test_split:]
    y_train=y[:train_test_split]
    return x_train,y_train,x_test,y_test

In [9]:
x_train,y_train,x_test,y_test=train_test_split(x,y,time_step=30)

In [10]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(96483, 30, 7) (96483, 1) (7263, 30, 7) (7263, 1)


In [11]:
save_data(x_train,y_train,'x_train','y_train')

In [12]:
save_data(x_test,y_test,'x_test','y_test')